In [84]:
# Import statements 
from sklearn.linear_model import LogisticRegression
import pandas as pd
from textblob import TextBlob
import string
import datetime

In [107]:
# Load data
train_feats = pd.read_csv('cleaned_feats/train_cleaned_features.csv', delimiter=',', encoding="utf-8")
dev_feats = pd.read_csv('cleaned_feats/dev_cleaned_features.csv', delimiter=',', encoding="utf-8")
test_feats = pd.read_csv('cleaned_feats/test_cleaned_features.csv', delimiter=',', encoding="utf-8")

# Clear whitespace
train_feats.columns = train_feats.columns.str.strip()

/opt/anaconda3/envs/prop-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [102]:
# Dropping stuff!

# Drop all rows where time is NA?
print("Old length: ", len(train_feats))
train_feats = train_feats[train_feats['time'].notna()]
print("New length: ", len(train_feats))

# Just drop hashtags for now, too difficult for M3 to deal with
train_feats = train_feats.drop(['hashtags'], axis=1)

Old length:  3515562
New length:  3515538


In [108]:
# View the dataframes
train_feats.head(5)
#train_feats.columns                                        

,text,place,user_location,followers_count,retweet_count,favorite_count,hashtags,weekday,month,day,time,year,sentiment
0,Pretty much any corona virus germ that wants t...,NaN,PG County,478,0,10,NaN,Thu,Mar,19.0,19:52:15,2020.0,positive
1,RT @RealSaavedra: Good.\n\nIt came from China.,NaN,"Sydney, New South Wales",394,102,0,NaN,Thu,Mar,19.0,19:52:15,2020.0,positive
2,RT @nicolebyer: Everyone in this looks sick,NaN,ferrets,695,48499,0,NaN,Thu,Mar,19.0,19:52:15,2020.0,negative
3,RT @jabinbotsford: Close up of President @real...,NaN,NaN,19,30492,0,NaN,Thu,Mar,19.0,19:52:15,2020.0,neutral
4,RT @Biancaixvi: Corona day 3: it just feels li...,NaN,"Huntley, IL",392,85302,0,NaN,Thu,Mar,19.0,19:52:15,2020.0,neutral


In [106]:
train_feats.dtypes

text                object
place                 int8
user_location        int32
followers_count     object
retweet_count       object
favorite_count      object
weekday               int8
month                 int8
day                float64
year               float64
sentiment             int8
hour                 int64
minute               int64
dtype: object

In [104]:
# Feature manipulation to make them ready for the model
# Change sentiment labels to numbers!
train_feats['sentiment'] = train_feats['sentiment'].astype('category')
train_feats["sentiment"] = train_feats["sentiment"].cat.codes

# Change location
train_feats["user_location"] = train_feats["user_location"].astype('category')
train_feats["user_location"] = train_feats["user_location"].cat.codes

# Change place
train_feats["place"] = train_feats["place"].astype('category')
train_feats["place"] = train_feats["place"].cat.codes

# Change weekday
train_feats["weekday"] = train_feats["weekday"].astype('category')
train_feats["weekday"] = train_feats["weekday"].cat.codes

# Change month
train_feats["month"] = train_feats["month"].astype('category')
train_feats["month"] = train_feats["month"].cat.codes

# Change time to datetime object
train_feats['time'] = train_feats['time'].apply(lambda x: datetime.datetime.strptime(str(x), '%H:%M:%S'))

# Add hour and minute columns
train_feats['hour'] = train_feats['time'].apply(lambda x: x.hour)
train_feats['minute'] = train_feats['time'].apply(lambda x: x.minute)

# Drop time
train_feats = train_feats.drop(['time'], axis=1)

# Split

In [25]:
# Haven't done text stuff yet!

print(test_feats.head(1)['text'][0])
blob = TextBlob(test_feats.head(1)['text'][0])
print(blob.tags)           # [('The', 'DT'), ('titular', 'JJ'),
                    #  ('threat', 'NN'), ('of', 'IN'), ...]

print(blob.noun_phrases)   # WordList(['titular threat', 'blob',
                    #            'ultimate movie monster',
                    #            'amoeba-like mass', ...])

for sentence in blob.sentences:
    print(sentence.sentiment.polarity)

@Jeff_Reinebold No corona out there, that’s for sure. Enjoy
[('@', 'NNS'), ('Jeff_Reinebold', 'NNP'), ('No', 'NNP'), ('corona', 'NN'), ('out', 'IN'), ('there', 'RB'), ('that', 'IN'), ('’', 'NNP'), ('s', 'NN'), ('for', 'IN'), ('sure', 'JJ'), ('Enjoy', 'NN')]
['jeff_reinebold', '’ s', 'enjoy']
0.5
0.4


In [12]:
# Instantiate model - Multinomial logistic regression
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')